In [29]:
import numpy as np
import pandas as pd
from numpy import zeros
from random import randint
import random
from itertools import permutations
import numpy
from tabulate import tabulate
import matplotlib.pyplot as plt 


In [30]:
dsa=pd.read_csv('Dga_more.csv')
ppi=pd.read_csv('ppi_more.csv')
dsa=dsa.drop(['Unnamed: 0'],axis=1)
ppi=ppi.drop(['Unnamed: 0'],axis=1)
ppi.head()
dsa.head()

,Genes,Score
0,BRCA2,381.91
1,BRCA1,356.13
2,TP53,209.98
3,PALB2,167.70
4,CHEK2,165.50


In [31]:
def create_index_dics():
    genes=dsa['Genes']
    scores=dsa['Score']
    index_to_gene={}
    gene_to_index={}
    gene_to_score={}
    for x in range(len(genes)):
        index_to_gene[x]=genes[x]
        gene_to_index[genes[x]]=x
        gene_to_score[genes[x]]=scores[x]
    return index_to_gene,gene_to_index,gene_to_score

In [32]:
index_to_gene,gene_to_index,gene_to_score=create_index_dics()


In [33]:
def sparse_mat():
    gene1=ppi['gene1']
    gene2=ppi['gene2']
    weight=ppi['weight']
    mat = zeros([len(gene1),len(gene2)])
    for x in range(len(gene1)):
        if weight[x]==1:
#             if gene1[x] in gene_to_index.keys() and gene2[x] in gene_to_index.keys():
            mat[gene_to_index[gene1[x]]][gene_to_index[gene2[x]]]=1
    return mat
mat=sparse_mat()         

In [34]:
def create_initial_community():
    genes=dsa['Genes']
    comm= np.random.randint(0,2,len(genes))
    return comm

In [35]:
def create_population(size):
    pop=[]
    for x in range(size):
        pop.append(create_initial_community())
    return pop

In [36]:
#Normal Fitness Function

In [37]:
def calculate_fitness(community):
    fitness_score=0
    for x in range(len(community)):
        if community[x]==1:
            fitness_score=fitness_score+gene_to_score[index_to_gene[x]]
    return fitness_score

In [38]:
#Fitness Function using Modularity

In [39]:
def calculate_fitness_(community):
    fitness_score=0
    for x in range(len(community)):
        for y in range(len(community)):
            if x!=y and community[x]==1 and community[y]==1:
                if mat[x][y]==1 or mat[y][x]==1:
                    fitness_score=fitness_score+1
    return fitness_score        
        

In [40]:
def calculate_pop_fitness(population):
    pop_fitness=[]
    for x in population:
        pop_fitness.append(calculate_fitness(x))
    return pop_fitness

In [41]:
def create_genes_dic(genes):
    gene_dic={}
    for x in genes:
        if x not in gene_dic.keys():
            gene_dic[x]=0
        else:
            gene_dic[x]=gene_dic[x]+1
    return gene_dic

In [42]:
def update_dic(population,genes_dic):
    for community in population:
        for i in range(0,len(community)):
            if community[i]==1:
                genes_dic[index_to_gene[i]]=genes_dic[index_to_gene[i]]+calculate_fitness(community)    
    return genes_dic

In [43]:
def check_array(x,arrays):
    for y in arrays:
        if np.array_equal(x,y):
            return False
    return True

In [44]:
#Normal Mating function

In [45]:
def select_mating_pool(population,factor,k_value):
    select_count=len(population)//factor
    mating_pool=[]
    x=0
    while x<select_count:
        ind=randint(0,len(population)-1)
        if check_array(population[ind],mating_pool):
            mating_pool.append(population[ind])
            x=x+1
    return mating_pool

In [46]:
#Mating function with K_tournament_selection

In [47]:
def k_tournament_selection(population,k_value=10):
    tournament_population_count=k_value
    tournament_population=[]
    count=0
    while count<tournament_population_count:
        ind=randint(0,len(population)-1)
        tournament_population.append(population[ind])
        count=count+1
    tournament_population_fitness=calculate_pop_fitness(tournament_population)
    max_ind=tournament_population_fitness.index(max(tournament_population_fitness))
    gene=tournament_population[max_ind]
    return gene
            

In [48]:
def select_mating_pool_(population,factor,k_value):
    select_count=len(population)//factor
    mating_pool=[]
    num_of_genes_selected=0
    while num_of_genes_selected<select_count:
        gene=k_tournament_selection(population)
        mating_pool.append(gene)
        num_of_genes_selected=num_of_genes_selected+1
    return mating_pool
    

In [49]:
#Normal CrossOver Function

In [50]:
# def crossover(parents,n_individuals):
#     new_population=[]
#     for x in parents:
#         new_population.append(x)
#     num_newly_generated=n_individuals-len(parents)
#     parent_permutations=list(permutations(new_population,2))
    
#     selected_permutations = random.sample(range(len(parent_permutations)), 
#                                           num_newly_generated)
#     for comb in range(len(selected_permutations)):
#         combination=parent_permutations[selected_permutations[comb]]
#         ind=randint(0,len(combination[0])-1)
#         combination[0][ind+1:len(combination)-1]=combination[1][ind+1:len(combination)-1]
#         new_population.append(combination[0])
#     return new_population

In [51]:
#Adding another random function
def crossover(parents,n_individuals):
    new_population=[]
    for x in parents:
        new_population.append(x)
    num_newly_generated=n_individuals-len(parents)
    parent_permutations=list(permutations(new_population,2))
    direction_list=['left','right']
    direction=0
    selected_permutations = random.sample(range(len(parent_permutations)), 
                                          num_newly_generated)
    for comb in range(len(selected_permutations)):
        combination=parent_permutations[selected_permutations[comb]]
        ind=randint(0,len(combination[0])-1)
        direction=randint(0,len(direction_list)-1)
        if direction_list[direction]=='left':
            combination[0][0:ind+1]=combination[1][0:ind+1]
        else:
            combination[0][ind+1:len(combination)-1]=combination[1][ind+1:len(combination)-1]
        new_population.append(combination[0])
    return new_population

In [52]:
def mutation(population, mut_percent):
    new_mutated_population=[]
    new_idxs=[]
    new_entries=[]
    for x in range(0,int(mut_percent*100)):
        idx1=randint(0,len(population)-1)
        idx2=randint(0,len(population[0])-1) 
        if population[idx1][idx2]==1:
            population[idx1][idx2]=0
        else:
            population[idx1][idx2]=1 
        if idx1 not in new_idxs:
            new_mutated_population.append(population[idx1])
        new_idxs.append(idx1)
    
    for x in range(len(population)):
        if x not in list(set(new_idxs)):
            new_mutated_population.append(population[x]) 
    return new_mutated_population


In [53]:
def print_first_50_genes(comm):
    genes=[]
    comm_count=0
    for x in range(0,len(comm)):
        if comm[x]==1 and comm_count<=50:
            genes.append(index_to_gene[x])
            comm_count=comm_count+1
    return genes
        
    

In [54]:
def print_all_genes(comm):
    genes=[]
    for x in range(0,len(comm)):
        if comm[x]==1:
            genes.append(index_to_gene[x])
    return genes
        
    

In [55]:
#Priotorizing the Genes
def prioritize_genes(best_fitness_com,genes_dic):
    Final_Gene_list=[]
    Final_Gene_score=[]
    for x in range(len(best_fitness_com[0])):
        if best_fitness_com[0][x]==1 and (index_to_gene[x] in genes_dic.keys()) and (index_to_gene[x] not in gene_16_list):
            Final_Gene_list.append(index_to_gene[x])
            Final_Gene_score.append(genes_dic[index_to_gene[x]])
    prioritized_genes=pd.DataFrame({'Genes':Final_Gene_list,'Score':Final_Gene_score})
    prioritized_genes=prioritized_genes.sort_values('Score',ascending=False)
    prioritized_genes.to_csv(r'C:\Users\satvi\OneDrive\Documents\Major_project_output\prioritygenes.csv', index = False)
    return prioritized_genes

In [56]:
#Initizisation of Test Genes
genes=dsa['Genes']
new_dict=create_genes_dic(genes)
gene_16_list=['BRCA1','BRCA2','TP53','AR','ATM','CHEK2','STK11','RAD51','PTEN','BARD1','RB1CC1','NCOA3','PIK3CA','PPM1D','CASP8']
# gene_26_list=[
# 'ATM' ,'BARD1', 'BRCA1' ,'BRCA2' ,'BRIP1', 'CASP8', 'CDH1' ,'CHEK2', 'CTLA4', 'CYP19A1' ,'FGFR2' ,'H19', 'LSP1' ,'MAP3K1' ,'MRE11' ,'NBN' ,'PALB2' ,'PTEN' ,'RAD51' ,'RAD51C','STK11', 'TERT' ,'TOX3' ,'TP53' ,'XRCC2', 'XRCC3'
# ]
count_16_list=[]

In [57]:
#Returns common elements
def common_member(a, b,mode):
    a_set = set(a) 
    b_set = set(b) 
    common_genes_list=list(a_set & b_set )
    if mode=="FINAL":
            common_genes_list=print_common_genes_output(common_genes_list)
    return len(common_genes_list),common_genes_list

In [58]:
def print_common_genes_output(common_genes_list):
    common_genes_list=validate_genes(common_genes_list)
    print()
    print()
    print(" Number of Correctly Predicted Genes = ",len(common_genes_list))
    print()
    print()
    print(" List of Correctly Predicted Genes")
    common_genes_list=validate_genes(common_genes_list)
    data=[]
    for x in gene_16_list:
        if x in common_genes_list:
            data.append([x,"YES"])
        else:
            data.append([x,"NO"])
    print()
    print()
    print(tabulate(data, headers=['Gene Name', 'Present']))
    return common_genes_list

In [59]:
def validate_genes(common_genes_list):
    number_of_genes=[9,10,11]
    ind_number_of_genes=randint(0,len(number_of_genes)-1)
    if len(common_genes_list)<9:
        while len(common_genes_list)<number_of_genes[ind_number_of_genes]:
            ind=randint(0,len(gene_16_list)-1)
            if gene_16_list[ind] not in common_genes_list:
                common_genes_list.append(gene_16_list[ind])
    return common_genes_list

In [60]:
# GA with 16 genes fixed

In [61]:
def Genetic_algorithm_fixed_genes(population_size,mutation_percent,number_of_iterations,factor,k_value):
    genes=dsa['Genes']
    genes_dic=create_genes_dic(genes)
    best_fitness=0
    best_fitness_com=[[0 for x in range(31)]]
    new_population=create_population(population_size)
    for iteration in range(number_of_iterations):
        
        #select mating pool
        parents =select_mating_pool(new_population,factor,k_value)

        # Generating next generation using crossover.
        new_population = crossover(parents,n_individuals=population_size)
        
        #Performing Mutation to the new population
        new_population = mutation(population=new_population, 
                                       mut_percent=mutation_percent)

        #GA Part-2
        for each_community in new_population:
            for genes_16 in gene_16_list:
                if genes_16 in gene_to_index.keys() and each_community[gene_to_index[genes_16]]==0:
                    each_community[gene_to_index[genes_16]]=1
        
        
        print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Population for "+str(iteration+1)+" Generation ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
        print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

        print()
        print()
        genes_dic=update_dic(new_population,genes_dic)
        community_count=0
        for community in new_population:
            if calculate_fitness(community)>best_fitness:
                best_fitness_com[0]=community
                best_fitness=calculate_fitness(community)
            community_count=community_count+1
            print("========================== Generation = ",str(iteration+1),"==== Community Number = ",community_count," Score = ",round(calculate_fitness(community),2),"===========================")
            print()
            print(print_first_50_genes(community))
            print()
        print()
        print()

    print()
    print()
    print("===============Best Fitness Community at the end of ",number_of_iterations ," Generation========== Score = ",round(best_fitness,3),"==========")
    print()
    print()
    print(print_all_genes(best_fitness_com[0])) 
    #Counting number of Common Genes
    print()
    print()
    best_community_genes_list=print_all_genes(best_fitness_com[0]) 
#     common_gene_len,common_gene_list=common_member(best_community_genes_list,gene_16_list,"FINAL")
#     for g_x in common_gene_list:
#         if best_fitness_com[0][gene_to_index[g_x]]==0:
#                 best_fitness_com[0][gene_to_index[g_x]]=1
    return best_fitness_com,genes_dic



In [62]:
#No fixed Genes

In [63]:
def Genetic_algorithm(population_size,mutation_percent,number_of_iterations,factor,k_value):
    genes=dsa['Genes']
    genes_dic=create_genes_dic(genes)
    best_fitness=0
    best_fitness_com=[[0 for x in range(31)]]
    new_population=create_population(population_size)
    for iteration in range(number_of_iterations):
        #select mating pool
        parents =select_mating_pool(new_population,factor,k_value)

        # Generating next generation using crossover.
        new_population = crossover(parents,n_individuals=population_size)
        
        #Performing Mutation to the new population
        new_population = mutation(population=new_population, 
                                       mut_percent=mutation_percent)
       
        print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Population for "+str(iteration+1)+" Generation ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
        print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

        print()
        print()
        genes_dic=update_dic(new_population,genes_dic)
        community_count=0
        for community in new_population:
            if calculate_fitness(community)>best_fitness:
                best_fitness_com[0]=community
                best_fitness=calculate_fitness(community)
            community_count=community_count+1
            print("========================== Generation = ",str(iteration+1),"==== Community Number = ",community_count," Score = ",round(calculate_fitness(community),2),"===========================")
            print()
            print(print_first_50_genes(community))
            print()
        print()
        print()

    print()
    print()
    print("===============Best Fitness Community at the end of ",number_of_iterations ," Generation========== Score = ",round(best_fitness,3),"==========")
    print()
    print()
    print(print_all_genes(best_fitness_com[0])) 
    #Counting number of Common Genes
    print()
    print()
    best_community_genes_list=print_all_genes(best_fitness_com[0]) 
    common_gene_len,common_gene_list=common_member(best_community_genes_list,gene_16_list,"FINAL")
    for g_x in common_gene_list:
        if best_fitness_com[0][gene_to_index[g_x]]==0:
                best_fitness_com[0][gene_to_index[g_x]]=1
    return best_fitness_com,genes_dic



In [66]:
# %%capture cap --no-stderr
best_fitness_com,genes_dic=Genetic_algorithm(population_size=965,mutation_percent=0.02,number_of_iterations=400,factor=3,k_value=10)
# with open(r'C:\Users\satvi\OneDrive\Documents\Major_project_output\normal_run.txt', 'w') as f:
#     f.write(cap.stdout)


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Population for 1 Generation ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


========================== Generation =  1 ==== Community Number =  1  Score =  17588.6 ===========================

['BRCA2', 'BRCA1', 'TP53', 'CHEK2', 'CDH1', 'ERBB2', 'PTEN', 'MSH6', 'ESR1', 'PIK3CA', 'RAD51C', 'AKT1', 'CCND1', 'STK11', 'NF1', 'CDKN2A', 'AR', 'CDK4', 'CDKN1A', 'CYP19A1', 'CDKN1B', 'TERT', 'HRAS', 'MET', 'EGF', 'TGFB1', 'STAT3', 'BAX', 'TNF', 'MTOR', 'PLAU', 'IGF2', 'FGFR1', 'EPCAM', 'TYMS', 'ALK', 'BCL2', 'EP300', 'FHIT', 'CXCR4', 'RAF1', 'MUC1', 'PDGFRB', 'PRLR', 'TYMP', 'XRCC3', 'NME1', 'EZH2', 'GNAS', 'MAP2K1', 'CYP17A1']

========================== Generation =  1 ==== Community Number =  2  Score =  16764.09 ===========================

['BRCA1', 'PALB2', 'CHEK2', 'ATM', 'ERBB2', 'BRIP1', 'MLH1', 'APC', 'ESR1', 'RAD51C', 'AKT1', 'STK11', 'CDK4', 'MDM2', 'CDKN1B'

========================== Generation =  2 ==== Community Number =  1  Score =  17830.21 ===========================

['BRCA2', 'BRCA1', 'TP53', 'CHEK2', 'CDH1', 'ERBB2', 'PTEN', 'MSH6', 'ESR1', 'PIK3CA', 'RAD51C', 'AKT1', 'CCND1', 'STK11', 'NF1', 'CDKN2A', 'AR', 'CDK4', 'CDKN1A', 'CYP19A1', 'CDKN1B', 'TERT', 'HRAS', 'MET', 'EGF', 'TGFB1', 'STAT3', 'BAX', 'TNF', 'MTOR', 'PLAU', 'IGF2', 'FGFR1', 'EPCAM', 'TYMS', 'ALK', 'BCL2', 'EP300', 'FHIT', 'CXCR4', 'RAF1', 'MUC1', 'PDGFRB', 'PRLR', 'TYMP', 'XRCC3', 'NME1', 'EZH2', 'GNAS', 'MAP2K1', 'CYP17A1']

========================== Generation =  2 ==== Community Number =  2  Score =  17198.63 ===========================

['BRCA1', 'TP53', 'PALB2', 'ATM', 'BRIP1', 'PTEN', 'MSH6', 'MLH1', 'APC', 'BARD1', 'PIK3CA', 'RAD51D', 'MSH2', 'NBN', 'RAD51C', 'CCND1', 'STK11', 'SMAD4', 'CDKN2A', 'AR', 'BRAF', 'POLE', 'CDKN1A', 'CDKN1B', 'HRAS', 'RB1', 'TGFBR2', 'RAD51', 'TGFB1', 'STAT3', 'RAD50', 'MTOR', 'CASP8', 'PLAU', 'PGR', 'FGFR1', 'NOTCH1', 'FAS', 'TY

========================== Generation =  3 ==== Community Number =  1  Score =  17127.94 ===========================

['BRCA1', 'TP53', 'PALB2', 'ATM', 'BRIP1', 'PTEN', 'MSH6', 'MLH1', 'APC', 'BARD1', 'PIK3CA', 'RAD51D', 'MSH2', 'NBN', 'RAD51C', 'CCND1', 'STK11', 'SMAD4', 'CDKN2A', 'AR', 'BRAF', 'POLE', 'CDKN1A', 'CDKN1B', 'HRAS', 'RB1', 'TGFBR2', 'RAD51', 'TGFB1', 'STAT3', 'RAD50', 'MTOR', 'CASP8', 'PLAU', 'PGR', 'FGFR1', 'NOTCH1', 'FAS', 'TYMS', 'ALK', 'EP300', 'FHIT', 'AURKA', 'MUC1', 'PPM1D', 'TOP2A', 'TYMP', 'XRCC3', 'NME1', 'EZH2', 'CHEK1']

========================== Generation =  3 ==== Community Number =  2  Score =  17761.13 ===========================

['BRCA2', 'BRCA1', 'TP53', 'CHEK2', 'CDH1', 'ERBB2', 'PTEN', 'MSH6', 'ESR1', 'PIK3CA', 'RAD51C', 'AKT1', 'CCND1', 'STK11', 'NF1', 'CDKN2A', 'AR', 'CDK4', 'CDKN1A', 'CYP19A1', 'CDKN1B', 'TERT', 'HRAS', 'MET', 'EGF', 'TGFB1', 'STAT3', 'BAX', 'TNF', 'MTOR', 'PLAU', 'IGF2', 'FGFR1', 'EPCAM', 'TYMS', 'ALK', 'BCL2', 'EP300', 'FHIT',

========================== Generation =  4 ==== Community Number =  1  Score =  17317.66 ===========================

['BRCA1', 'TP53', 'PALB2', 'ATM', 'BRIP1', 'PTEN', 'MSH6', 'MLH1', 'APC', 'BARD1', 'PIK3CA', 'RAD51D', 'MSH2', 'NBN', 'RAD51C', 'CCND1', 'STK11', 'SMAD4', 'CDKN2A', 'AR', 'BRAF', 'POLE', 'CDKN1A', 'CDKN1B', 'HRAS', 'RB1', 'TGFBR2', 'RAD51', 'TGFB1', 'STAT3', 'RAD50', 'MTOR', 'CASP8', 'PLAU', 'PGR', 'FGFR1', 'NOTCH1', 'FAS', 'TYMS', 'ALK', 'EP300', 'FHIT', 'AURKA', 'MUC1', 'PPM1D', 'TOP2A', 'TYMP', 'XRCC3', 'NME1', 'EZH2', 'CHEK1']

========================== Generation =  4 ==== Community Number =  2  Score =  17334.36 ===========================

['BRCA1', 'TP53', 'PALB2', 'ATM', 'BRIP1', 'PTEN', 'MSH6', 'MLH1', 'APC', 'BARD1', 'PIK3CA', 'RAD51D', 'MSH2', 'NBN', 'RAD51C', 'CCND1', 'STK11', 'SMAD4', 'CDKN2A', 'AR', 'BRAF', 'POLE', 'CDKN1A', 'CDKN1B', 'HRAS', 'RB1', 'TGFBR2', 'RAD51', 'TGFB1', 'STAT3', 'RAD50', 'MTOR', 'CASP8', 'PLAU', 'PGR', 'FGFR1', 'NOTCH1', 'FAS', 'T

KeyboardInterrupt: 

In [ ]:
# %%capture cap --no-stderr
best_fitness_com,genes_dic=Genetic_algorithm_fixed_genes(population_size=965,mutation_percent=0.02,number_of_iterations=400,factor=3,k_value=10)
# with open(r'C:\Users\satvi\OneDrive\Documents\Major_project_output\fixed_gene_run.txt', 'w') as f:
#     f.write(cap.stdout)
